In [26]:
import io
import sys
import numpy as np
import matplotlib.pyplot as plt
plt.figure(figsize=(16,9))
from mpl_toolkits.axes_grid1 import make_axes_locatable
from collections import OrderedDict
import IPython
import pytz
import datetime
import json
import logging
!python3 --version

Python 3.8.5


<Figure size 1152x648 with 0 Axes>

# Vader Sentiment

```python
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer  
analyser = SentimentIntensityAnalyzer()
def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
    print("{:-<40} {}".format(sentence, str(score)))
```

# GeoPandas
<a href="https://geopandas.org"><img src="img/l_geopandas.png" style="float: right;"></a>
  
```python
!pip3 install geopandas
import geopandas as gpd
geo_df = geopandas.read_file("data/maps/usgeojson/gz_2010_us_040_00_5m.json")
geo_df.head()
geo_df = gpd.read_file("data/maps/states_21basic/states.shp")
geo_df["STATE_FIPS"] = geo_df["STATE_FIPS"].astype(np.int64)
geo_df.head()
geo_df.dtypes
geo_df.plot()
df = pd.read_csv("data/uspop-nst-2018.csv", header=0)
df = df[['STATE_FIPS','POP_2018']]
merged = geo_df.join(df.set_index("STATE_FIPS"), on="STATE_FIPS")
merged.head()
fig, ax = plt.subplots(1, 1)
divider = make_axes_locatable(ax)
merged.plot(column='POP_2018',
           ax=ax,
           legend=True,
           legend_kwds={'label': "Population by State (m)",
                        'orientation': "horizontal"})
merged.plot(column='POP_2018');
cax = divider.append_axes("right", size="5%", pad=0.1)
merged.plot(column='POP_2018', ax=ax, legend=True, cax=cax)

fig.savefig("leaddistribution.png", dpi=300)
```


# Excel<a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html"><img src="img/l_excel.png" style="float: right;"></a>     
  
```python
import pandas as pd
url ='http://s3.amazonaws.com/assets.datacamp.com/course/importing_data_into_r/latitude.xls'
xl = pd.read_excel(url,sheetname=None)
print(xl.keys())
print(xl['1700'].head())


!pip install xlrd
import xlrd
xlsx = pd.ExcelFile('examples/ex1.xlsx')
df = pd.read_excel('examples/ex1.xlsx', 'Sheet1')
writer = pd.ExcelWriter('examples/ex2.xlsx')
frame.to_excel(writer, 'Sheet1')
df.to_excel('examples/ex2.xlsx')
df.to_excel(writer, 'Sheet1')
writer.save()
!rm examples/ex2.xlsx
```
--------------------------------------

# BeautyfulSoup4
<a href="https://www.crummy.com/software/BeautifulSoup/"><img src="img/l_beautsoup4.jpg" style="width: 50px; float: right;"></a>

```python
!pip3 install BeautifulSoup4

Scape all URL's off a page
import requests
from bs4 import BeautifulSoup
url = 'https://www.python.org/~guido/'
r = requests.get(url)
html_doc = r.text
soup = BeautifulSoup(html_doc)
print(soup.title)
a_tags = soup.find_all('a')
for link in a_tags:
    print(link.get('href'))
```

In [39]:

# Import packages
import requests
from bs4 import BeautifulSoup
# Specify url: url
url = 'https://www.python.org/~guido/'
# Package the request, send the request and catch the response: r
r = requests.get(url)
# Extracts the response as html: html_doc
html_doc = r.text
# Create a BeautifulSoup object from the HTML: soup
soup = BeautifulSoup(html_doc)
# Prettify the BeautifulSoup object: pretty_soup
pretty_soup =  soup.prettify()
# Print the response
print(pretty_soup)

<html>
 <head>
  <title>
   Guido's Personal Home Page
  </title>
 </head>
 <body bgcolor="#FFFFFF" text="#000000">
  <h1>
   <a href="pics.html">
    <img border="0" src="images/IMG_2192.jpg"/>
   </a>
   Guido van Rossum - Personal Home Page
   <a href="pics.html">
    <img border="0" height="216" src="images/guido-headshot-2019.jpg" width="270"/>
   </a>
  </h1>
  <p>
   <a href="http://www.washingtonpost.com/wp-srv/business/longterm/microsoft/stories/1998/raymond120398.htm">
    <i>
     "Gawky and proud of it."
    </i>
   </a>
  </p>
  <h3>
   <a href="images/df20000406.jpg">
    Who I Am
   </a>
  </h3>
  <p>
   Read
my
   <a href="http://neopythonic.blogspot.com/2016/04/kings-day-speech.html">
    "King's
Day Speech"
   </a>
   for some inspiration.
  </p>
  <p>
   I am the author of the
   <a href="http://www.python.org">
    Python
   </a>
   programming language.  See also my
   <a href="Resume.html">
    resume
   </a>
   and my
   <a href="Publications.html">
    publicati

In [32]:
# Import package
import requests
# Specify the url: url
url = "http://www.datacamp.com/teach/documentation"
# Packages the request, send the request and catch the response: r
r= requests.get(url)
# Extract the response: text
text = r.text
# Print the html
print(text)

57 8.018 12 5.375 10.643 4z" />
            </svg>
          </a>
      </form>
      </div>
    </div>
  </div>
</header>

    <div class="container">
      <div class="content educate_content"><section class="section">
    <div class="g__space">
      <a href="/en/collections/1256585-getting-started" class="paper ">
        <div class="collection o__ltr">
          <div class="collection__photo">
            <svg role='img' viewBox='0 0 48 48'><g id="chat-star" stroke-width="2" fill="none" fill-rule="evenodd" stroke-linejoin="round"><path d="M20 34.942c-2.083-.12-4.292-.42-6-.942L3 39l4-9c-3.858-3.086-6-7.246-6-12C1 8.61 10.328 1 21.835 1 33.343 1 43 8.61 43 18c0 1.044-.117 2.065-.342 3.057"></path><path d="M36.016 25L40 33h7l-6 5 3 9-8-5.494L28 47l3-9-6-5h7l4.016-8z"></path></g></svg>
          </div>
          <div class="collection_meta" dir="ltr">
            <h2 class="t__h3 c__primary">Getting Started</h2>
            <p class="paper__preview">Everything you need to know to beg

In [33]:
# Import packages
import requests
from bs4 import BeautifulSoup
# Specify url: url
url = 'https://www.python.org/~guido/'
# Package the request, send the request and catch the response: r
r = requests.get(url)
# Extracts the response as html: html_doc
html_doc = r.text
# Create a BeautifulSoup object from the HTML: soup
soup = BeautifulSoup(html_doc)
# Prettify the BeautifulSoup object: pretty_soup
pretty_soup =  soup.prettify()
# Print the response
print(pretty_soup)

ModuleNotFoundError: No module named 'bs4'

In [31]:
# Import packages
from urllib.request import urlopen, Request
# Specify the url
url = "http://www.datacamp.com/teach/documentation"
# This packages the request
request = Request(url)
# Sends the request and catches the response: response
response = urlopen(request)
# Extract the response: html
html = response.read()
# Print the html
print(html)
# Be polite and close the response!
response.close()

HTTPError: HTTP Error 403: Forbidden

In [30]:
# Import packages
from urllib.request import urlopen, Request
# Specify the url
url = "http://www.datacamp.com/teach/documentation"
# This packages the request: request
request=Request(url)
# Sends the request and catches the response: response
response = urlopen(request)
# Print the datatype of response
print(type(response))
# Be polite and close the response!
response.close()

HTTPError: HTTP Error 403: Forbidden

# Scrapy
<a href="https://scrapy.org"><img src="img/l_scrapy.png" style="width: 200px; float: right;"></a>  

  
```python
!pip3 install scrapy
!pip3 install json
!pip3 install logging

import pandas as pd
import scrapy
from scrapy.crawler import CrawlerProcess
import json
import logging
class JsonWriterPipeline(object):
    def open_spider(self, spider):
        self.file = open('venues.jl', 'w')
    def close_spider(self, spider):
        self.file.close()
    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item
class QuotesSpider(scrapy.Spider):
    name = "venues"
    start_urls = ['https://www.cvent.com/venues/results/United%20States?']
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1},       # Used for pipeline 1
        'FEED_FORMAT':'json',                                       # Used for pipeline 2
        'FEED_URI': 'venues.json'                                   # Used for pipeline 2
    }
    def parse(self, response):
        for venue in response.css('div.quote'):
            yield {
                'name': quote.css('span.text::text').extract_first(),           
data-cvent-id="searchResult-mainContent-mainResults-venueListContainer-ul-li-0-venueCard-link-wrapper-venueInfoWrapper-venueName
                'author': quote.css('span small::text').extract_first(),
                'tags': quote.css('div.tags a.tag::text').extract(),
            }
        NEXT_PAGE_SELECTOR = '.next a ::attr(href)'
        next_page = response.css(NEXT_PAGE_SELECTOR).extract_first()
        if next_page:
            yield scrapy.Request(
                response.urljoin(next_page),
                callback=self.parse
            )
```


# Bokah
<a href="https://www.mysql.com/products/connector/"><img src="img/l_bokeh.png" style="float: right;"></a>  
  
```python
!pip3 install bokeh
!pip3 install hvplot
import bokeh  
from bokeh.io import output_notebook, output_file, show, push_notebook
from bokeh.plotting import *
from bokeh.models import ColumnDataSource, HoverTool, CategoricalColorMapper
from bokeh.layouts import row, column, gridplot, widgetbox
from bokeh.models.widgets import Tabs, Panel
from bokeh.transform import factor_cmap
import hvplot as hv
import hvplot.pandas  
```


# MySQL
<a href="https://www.mysql.com/products/connector/"><img src="img/l_mysql.png" style="float: right;"></a>   
  
```python
!pip install mysql-connector-python
import mysql.connector
config = {
    'host': 'rpsmithii.mysql.pythonanywhere-services.com',
    'database': 'rpsmithii$weight','user': 'rpsmithii',
    'password': 'Home@Call4','port': '3306'}
db = mysql.connector.connect(**config)
cur = db.cursor()
cur.execute("SELECT dt, wht FROM weight WHERE wht > 10")
table = pd.DataFrame(cur.fetchall())
table.columns = cur.column_names
```


# Pandas
<a href="https://pandas.pydata.org"><img src="img/l_pandas.png" style="float: right;"></a> 
  
```python
!pip3 install pandas
import pandas as pd                     
```

### csv methods
```python
df = pd.read_csv('data/hosp.csv', index_col='Target?', dtype={'user_id': int})
with open('csvfile.csv', 'w') as csvfile: f = csv.writer(csvfile) f.writerows(items)   # write to csv file
```

### attributes
```python
pd.info()                               # data types and specs
pd.shape                                # dimensions (tuple)
pd.describe()                           # shows a quick statistic data summary
pd.dtypes                               # column labels & data types
pd.index                                # index (row labels)
pd.head(n)                              # return first n rows
pd.tail(n)                              # return last n rows    
pd.columns                              # column labels
pd.values                               # Numpy representation
pd.axes                                 # list axes
pd.size                                 # number (int) elements
pd.memory_usage([index, deep])          # each column memory (bytes)
```

### transforms
```python
pd.between_time(start_time, end_time)	# select range
pd.set_index()                          # set index using existing columns
pd.['col0'].astype(str)                 # change data type
df = pd.drop("del", axis=0)             # delete all rows with label "del"
df = pd.concat([df01, df02, df03])      # combine dataframes
pd.is_copy                              # return copy
pd.empty                                # empty
df = df0.drop(columns=[' X', ' N'])     # drop columns
df = df0.sort_values(by='column')       # sort
```

### [plots](https://pandas.pydata.org/pandas-docs/stable/user_guide/visualization.html)

```python
plt.figure(figsize=(16,9))
plt.rcParams["figure.figsize"] = [16,9]
df.plot.area([x, y])                    # stacked area plot
df.plot.bar([x, y])                     # vertical bar plot
df.plot.barh([x, y])                    # horizontal bar plot
df.plot.box([by])                       # box plot
df.boxplot([column, by, ax, …])         # box plot from columns
df.plot.hist([by, bins])                # histogram of columns
df.hist([column, by, grid, …])          # histogram
df.plot.line([x, y])                    # columns as lines
df.plot.pie([y])                        # pie plot
df.plot.scatter(x, y[, s, c])           # scatter plot
df.plot(legend=False)
df.plot(xlabel="new x", ylabel="new y")
ts.plot(logy=True)                      # log-scale
df['B'].plot(secondary_y=True, style='g') # secondary axis
df['A'].plot(x_compat=True)             # compa
df.plot.line()
with pd.plotting.plot_params.use('x_compat', True):
    df[' Air Temperature'].plot(color='b')
    df[' Water Temperature'].plot(color='g')
plt.rcParams["figure.figsize"] = [16,9]
dfa.plot.line()
dfw.plot.line()
```

### lookup()
Extract a set of values given a sequence of row labels and column labels return **NumPy array**

```python
DataFrame.lookup(list(range(0, 10, 2)), ['B', 'C', 'A', 'B', 'D'])
```

### query()

column b has values between column a and c values
```python
DataFrame.query('(a < b) & (b < c)')
index = pd.MultiIndex.from_arrays([colors, foods], names=['color', 'food'])
```

columns a and "b" have overlapping values
```python
DataFrame.query('a in b')
```

columns a and b have overlapping values and col c's values are less than col d's
```python
DataFrame.query('a in b and c < d')
```

Comparing a list of values to a column using ==/!= works similarly to in/not in.
```python
DataFrame.query('b == ["a", "b", "c"]')
```

select rows with index values 'Andrade' + 'Veness' with columns fr 'city' to 'email'  
```python
nDataFrame.loc[['Andrade', 'Veness'], 'city':'email']
```  
  
select same rows, with just 'first_name', 'address' and 'city' columns  
```python
DataFrame.loc['Andrade':'Veness', ['first_name', 'address', 'city']]
```  
  
select rows with _first name_ Antonio and _columns_ 'city' to 'email'  
  
```python
DataFrame.loc[DataFrame['first_name'] == 'Antonio', 'city':'email']
```  
  
Select rows where email column ends w/ 'hotmail.com' include all columns  
```python
DataFrame.loc[DataFrame['email'].str.endswith("hotmail.com")]
```  

--------------------------------------

# Zipcode Methods  
  
```python
!pip3 install uszipcode
!pip3 install --upgrade uszipcode                # upgrade databases

from uszipcode import SearchEngine
search = SearchEngine(simple_zipcode=False)      # set simple_zipcode=False to use rich info database
search = SearchEngine(simple_zipcode=True)       # False uses rich info database
zipcode = search.by_zipcode("06916")
zipcode.to_json()                                # to json
zipcode.to_dict()                                # to dict
zip = zipcode.values()                           # to list
zipcode
```
----------------------------------

In [ ]:
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}